In [7]:
import random
import operator
from datetime import datetime
import copy

In [17]:

class ant:
    
    def __init__(self,noNodes):
        
        self.__path=[random.randint(0,noNodes-1)] # we start from a random node
        self.__noNodes = noNodes #number of nodes in graph
    
    @property
    def path(self):
        return self.__path
    
    #input: pheromone - quantity of pheromone from i to j    distance - the cost of the ij edge
    #output: float representing the score of this path
    def calculate(self,pheromone,distance):
        return pow(pheromone,3)*pow((1/distance),2) # we use pow() to control what is more important in the decision making process.
                                                    # in this case we use pheromone as the dominant factor
    
    #input: probs - list of floats representing the probability to pick a certain node as the next one
    #       univisited - list containing all the unvisited nodes
    #output: the next node to be visited
    # how pick works:
    #     probs = [0.7,0.2,0.1]
    #     unvisited = [1,2,3]   => 1 is between 1 and 0.2+0.1 = 0.3
    #                              2 is between 0.3 and 0.1
    #                              3 is between 0.1 and 0
    # we pick a random number between 0 and 1,then check in what interval it fits and return the node corresponding to that interval
    # the implementation is up to everyone                                     
    def pick(self,probs,unvisited):
        
        pr = {}  
        for i in range(len(probs)):
            pr[unvisited[i]]=probs[i]          
        pr = sorted(pr.items(), key=operator.itemgetter(1), reverse=True)
        probs = sorted(probs,reverse=True)
        
        choice = random.random()
        
        
        upper=1
        lower=sum(probs[1:])
        for i in range(0,len(probs)+1):
            #print(choice," ",upper," ",lower)
            if choice<=upper and choice>=lower:
                #print(pr[i][0])
                if i==len(probs):
                    return pr[-1][0]
                else:
                    return pr[i][0]
            else:
                upper=lower
                if i==len(probs):
                    return pr[-1][0]
                else:
                    lower=sum(probs[i+1:])
            
        
            
    #input: adj -  adjacency matrix       pheromoneMatrix - pheromone matrix          q0 - float(a probability)
    #output: the new visited node
    def move(self,adj,pheromoneMatrix,q0):
        curent = self.__path[-1]
        q = random.random();
        
        
        unvisited=[]
        for i in range(0,self.__noNodes):
            if i not in self.__path:
                unvisited.append(i)
              
        next=0
        if q<=q0: # if our probability is less or equal than the given one we just calculate the best path and pick it
            max=0    
            for i in range(len(unvisited)):
                #print(pheromoneMatrix[curent][unvisited[i]]," ",int(adj[curent][unvisited[i]]))
                rez = self.calculate(pheromoneMatrix[curent][unvisited[i]],int(adj[curent][unvisited[i]]))
                if rez >=max:
                    max=rez
                    next=unvisited[i]
                #print(rez)
            self.__path.append(next)
            unvisited.remove(next)
            
        else: # else we calculate the probability of all possible paths and pick one of them using pick() function
            probs =[]
            for i in range(len(unvisited)):
                rez = self.calculate(pheromoneMatrix[curent][unvisited[i]],int(adj[curent][unvisited[i]]))
                probs.append(rez)
            suma = sum(probs)
            for i in range(len(unvisited)):
                probs[i]/=suma # normalizing hte probabilities
            
            next = self.pick(probs,unvisited)
            
            self.__path.append(next)
            
            unvisited.remove(next) # in both scenarios we pick a node that we have te remove from unvisited
            
        pheromoneMatrix[curent][next]+=1/int(adj[curent][next]) # update the pheromone matrix for the path we took to help 
                                                                # future ants in their picking decision
        return next 
        
        
        
    def __str__(self):
        return "\n ant: "+str(self.__path)
    

In [18]:
class ACO:
    def __init__(self,param):
        self.__param = param
        self.__adj=param["adj"] # adjacency matrix
        self.__pheromoneMatrix=param["pheromoneMatrix"] # pheromone matrix
        self.__population = self.initialization() # population of param['popSize'] ants
        
    @property
    def population(self):
        return self.__population
    @property
    def pheromoneMatrix(self):
        return self.__pheromoneMatrix
    
    def distance(self,path): # calculates the total distance of the path given
        sum=0
        for i in range(0,len(path)):
            sum+=int(self.__adj[path[i-1]][path[i]])
        return sum
    
    def bestAnt(self): # returns the best ant form this population
        best=self.__population[0]
        bestDistance=self.distance(best.path)
        for i in self.__population:
            if self.distance(best.path)>=self.distance(i.path):
                best=i
        return best
            
        
    def initialization(self):
        rez=[]
        for i in range(self.__param["popSize"]):
            rez.append(ant(self.__param["noNodes"]))
        #print(self.__param["popSize"])
        return rez
    
    def updatepheromoneMatrix(self,path):# updates the pheromone matrix on the path given
        for i in range(len(path)-1):
            self.__pheromoneMatrix[i][i+1]+=1/self.distance(self.bestAnt().path)
    
    def journey(self,q0): # q0 is a probability that indivates how likely is the graph to change during ant journeys
        self.__population=self.initialization()
        
        q=random.random()
        
        if(q<q0): # in this case the graph changes randomly and our ants should adapt the result 
            noduri=random.randint(1,5)
            for _ in range(noduri):
                i=random.randint(1,self.__param["noNodes"]-1)
                j=random.randint(1,self.__param["noNodes"]-2)
                if i==j:
                    j+=1
                
                distance=random.randint(5,35)
                if noduri<=4:
                    
                    self.__adj[i][j]=str(int(self.__adj[i][j])+distance)
                    self.__pheromoneMatrix[i][j]-=(self.__pheromoneMatrix[i][j]*pow((1/distance),2))
                else:
                    
                    self.__adj[i][j]=str(int(self.__adj[i][j])-random.randint(5,20))
                    self.__pheromoneMatrix[i][j]-=(self.__pheromoneMatrix[i][j]*pow((1/distance),2))
                
        # we can use 2 strategyes now.
        # 1: every ant modifyes the pheromone matrix at every step (fast converging method) - best for big graphs
        # 2: at the end of a journey only the best ant modifyes the pheromone matrix (slow and steady method) - best for small graphs
        
        localpheromoneMatrix=self.__param['pheromoneMatrix'];   #in case of biger graphs
        if self.__param['noNodes']<=12:
            localpheromoneMatrix=copy.deepcopy(self.__param['pheromoneMatrix']);  #in case of smaller graphs we create a copy of
                                                                                # of pheromone matrix to pass to the move function
            
        for _ in range(self.__param["noNodes"]-1):
            for i in range(len(self.__population)):
                self.__population[i].move(self.__adj,localpheromoneMatrix,self.__param['probability'])
        
        self.updatepheromoneMatrix(self.bestAnt().path) # in both cases the best and updates the pheromone matrix
        #print(self.__pheromoneMatrix,'\n')
        return self.distance(self.bestAnt().path)
    
    def __str__(self):
        return "\n ACO: "+ self.__population

In [19]:
#provides a dictionary with all needed data
def data(filename):
    f=open(filename,"r")
    a=f.read()
    a=a.split()

    mat=[]
    for i in range(1,int(a[0])+1): 
        mat.append(a[i].split(",")) 
    param={}
    param["adj"]=mat
    param["probability"]=0.5
    param["popSize"]=len(mat)-1
    param["noNodes"]=len(mat)
    param["pheromoneMatrix"]=[]
    
    for i in range(len(param["adj"])):
        list=[]
        for j in range(len(param["adj"])):
            if i==j:
                list.append(0)
            else:
                list.append(1)
        param["pheromoneMatrix"].append(list)
    
    
    
    
   
    
    return param
    

In [20]:

def main(filename,numberOfSteps, dynamicGraph):  # set dynamicGraph to zero if the graph should be static
    params = data(filename)
    aco  = ACO(params)
    
    #print(aco.distance([0,1,2,3,4]))
    best=9999
    for _ in range(numberOfSteps):
        a = aco.journey(dynamicGraph)
        if a<best:
            best=a
    return best


In [27]:
#measuring how accurate is the alg... asuming we know the answer
# 17.txt = 2085

def accuracy():
    print(datetime.now())
    c=0
    for _ in range(100):
        a=main("26.txt",200,0.00)
        if(a==2085):
            c+=1
        print(a)
    print(str(c)+"%")
    print(datetime.now())
accuracy()

2021-04-14 13:33:26.947101
937
0%
2021-04-14 13:33:32.244921


In [ ]:
# The ACO alg is much more likely to give the best result compared to GA because it builds it`s result gradually based on
#previous choices. So in other words it uses a mix of heuristics and random operations wich can lead better to the overall best result.

#use example
print(main("26.txt",200,0.01))